In [45]:
import pandas as pd
from scipy.spatial import distance
from sklearn.manifold import TSNE
import plotly.express as px

pd.set_option("display.float_format", lambda x: "%.10f" % x)

## Clean data for energy profile comparison

In [46]:
# Import data and rename columns for better readability
data = pd.read_csv("Data/annual_generation_state.csv", thousands=",")
data.rename(
    columns={
        "TYPE OF PRODUCER": "PRODUCER",
        "ENERGY SOURCE": "SOURCE",
        "GENERATION (Megawatthours)": "MWH",
    },
    inplace=True,
)

# Change all strings to upper for consistency (state column has 'US-Total' and 'US-TOTAL')
data["STATE"] = data["STATE"].str.upper()
data["PRODUCER"] = data["PRODUCER"].str.upper()
data["SOURCE"] = data["SOURCE"].str.upper()

# Filter data to not include 'US-TOTAL' 
data =  data[data['STATE'] != 'US-TOTAL']

# Remove the blank state rows (3 rows for 2003 with 0 generation in coal, natural gas, and petroleum)
data = data[data['STATE'] != '  ']


# Filter data for 2016 and only include 'Total Electric Power Industry'
data_2016 = data[
    (data["YEAR"] == 2016) & (data["PRODUCER"] == "TOTAL ELECTRIC POWER INDUSTRY")
]

# Group and pivot the data to have energy sources as columns
data_2016 = (
    data_2016.groupby(["STATE", "SOURCE"])["MWH"].sum().unstack("SOURCE", fill_value=0)
)

## Calculate the Euclidean distance between the energy profiles to find those nearest to Iowa's

In [47]:
# Compute the Euclidean distance between each state with reference to Iowa
distances = {}
for state, row in data_2016.iterrows():
    distances[state] = distance.euclidean(data_2016.loc["IA"], row)

# Sort the states by distance from Iowa
sorted_states = sorted(distances.items(), key=lambda x: x[1])

# Print the top 10 states with similar energy profiles to Iowa
print("Top 10 states with similar energy profiles to Iowa:")
for i in range(10):
    e_profile = data_2016.loc[sorted_states[i][0]]
    print(f"State: {sorted_states[i][0]}, Distance: {sorted_states[i][1]}")

Top 10 states with similar energy profiles to Iowa:
State: IA, Distance: 0.0
State: KS, Distance: 10020219.82960803
State: CO, Distance: 15938323.087116223
State: MN, Distance: 15939496.711649116
State: ND, Distance: 21065706.51148387
State: WY, Distance: 23510794.41443885
State: NE, Distance: 24951273.514443345
State: UT, Distance: 26303915.562379833
State: WI, Distance: 26711262.876487963
State: AR, Distance: 27523239.5540178


## Visualize the Euclidean distances using t-SNE to reduce dimensionality

In [49]:
# Use t-SNE to reduce dimensionality to 3D
tsne = TSNE(n_components=3, random_state=42)
reduced_data_tsne = tsne.fit_transform(data_2016)

# Create a dataframe with the reduced data
reduced_df_tsne = pd.DataFrame(reduced_data_tsne, columns=["x", "y", "z"])
reduced_df_tsne.index = data_2016.index

# Add the distance column to the dataframe and sort
reduced_df_tsne["DISTANCE_TO_IA"] = reduced_df_tsne.index.map(lambda x: distances[x])
reduced_df_tsne.sort_values("DISTANCE_TO_IA", inplace=True)

# Closer to IA -> Yellow, Farther from IA -> Red, IA itself -> Green
color_scale = [(0, "green"), (0.1, "yellow"), (.5, "orange"), (1, "red")]

# Create interactive plotly scatter plot
fig = px.scatter_3d(
    reduced_df_tsne,
    x="x",
    y="y",
    z="z",
    text=reduced_df_tsne.index,
    hover_name=reduced_df_tsne.index,
    color="DISTANCE_TO_IA",
    color_continuous_scale=color_scale,
    opacity=0.8,
    labels={
        "x": "t-SNE 1",
        "y": "t-SNE 2",
        "z": "t-SNE 3",
        "DISTANCE_TO_IA": "Energy Profile Distance to IA",
    },
    title="t-SNE of Energy Sources Profiles in 2016",
)

fig.update_traces(marker=dict(size=3), textfont=dict(size=5))
fig.show()